##  Does the UMich Consumer Sentiment Index interact with people's political perceptions ?

### Where the data come from 
- At this point I am not sure how to get it API or even by URL 
- There is a funny UX for Cross Sectional Data Archive at https://sda.umsurvey.org/sda-public/cgi-bin/hsda?harcsda+sca

Where you have go and manually set parameters, which is not quite reproducible 
- I did `row = year_month, column = ics, controll=polaff`
- then `Run the Table` and downloaded a file 

In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
ICS_POL = pd.read_csv(f"AAQrGIHa.csv")
ICS_POL

In [ ]:
ICS_POL.dtypes

In the accompanying code book it says that POLAFF has the following coding:


>Text of this Question or Item
>Generally speaking, do you usually think of yourself as a Republican, a Democrat, an Independent, orwhat?

>Category Labels:

> - 1:	Republican
> - 2:	Democrat
> - 3:	Independent, No Preference
> - 8:	DK
> - 9:	NA 

In [ ]:
# Convert POLAFF to numeric values
ICS_POL['POLAFF'] = pd.to_numeric(ICS_POL['POLAFF'], errors='coerce')

# Convert ICS to numeric values
ICS_POL['ICS'] = pd.to_numeric(ICS_POL['ICS'], errors='coerce')

Compute average ICS for Republican (POLAFF = 1), Democrat (POLAFF = 2), Independent (POLAFF = 3), and overall respondents per YYYYMM

In [ ]:

ics_rep = ICS_POL[ICS_POL['POLAFF'] == 1].groupby('YYYYMM')['ICS'].mean().rename('ICS_rep')
ics_dem = ICS_POL[ICS_POL['POLAFF'] == 2].groupby('YYYYMM')['ICS'].mean().rename('ICS_dem')
ics_ind = ICS_POL[ICS_POL['POLAFF'] == 3].groupby('YYYYMM')['ICS'].mean().rename('ICS_ind')
ics_overall = ICS_POL.groupby('YYYYMM')['ICS'].mean().rename('ICS_overall')

# Merge results into a new dataframe
ics_summary = pd.concat([ics_rep, ics_dem, ics_ind, ics_overall], axis=1).reset_index()

# Display the cleaned dataset
ics_summary

Translate to data to a Pandas Datetime Object 

In [ ]:
ics_summary['YYYYMM'] = pd.to_datetime(ics_summary['YYYYMM'], format='%Y%m')

In [ ]:
# drop rows with NaN values
ics_summary = ics_summary.dropna()
ics_summary 

In [ ]:
#Lets go with after 2006
ics_summary = ics_summary[ics_summary['YYYYMM'] > '2006-01-01']

In [ ]:
fig = px.line(ics_summary, x='YYYYMM', y=['ICS_rep', 'ICS_dem', 'ICS_ind', 'ICS_overall'], title='Average ICS by Political Affiliation')
fig.show()






In [ ]:
# redo plot with ICS_rep = red, ICS_dem = blue, ICS_ind = green, ICS_overall = black
#put a vertical line at 2017-1-20, 2021-1-20, 2025-1-20
fig = px.line(ics_summary, x='YYYYMM', y=['ICS_rep', 'ICS_dem', 'ICS_ind', 'ICS_overall'], title='Average Index of Consumer Sentiment by Political Affiliation')
fig.update_traces(line=dict(width=2))
fig.update_traces(line=dict(width=2))
fig.update_traces(line_color='black', selector=dict(name='ICS_overall'))
fig.update_traces(line_color='red', selector=dict(name='Republican'))
fig.update_traces(line_color='blue', selector=dict(name='Democrat'))
fig.update_traces(line_color='green', selector=dict(name='Independent'))
fig.add_vline(x='2008-11-05', line_width=2, line_dash="dash", line_color="red")
fig.add_vline(x='2016-11-05', line_width=2, line_dash="dash", line_color="red")
fig.add_vline(x='2020-11-05', line_width=2, line_dash="dash", line_color="red")
fig.add_vline(x='2024-11-05', line_width=2, line_dash="dash", line_color="red")


